In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, widgets
from plotly.validators.scatter.marker import SymbolValidator
from sklearn.metrics import r2_score
import math

In [2]:
ex_data1 = pd.read_csv('Pt_data_constantpotential.csv')

ex_data = pd.read_csv('Pt_data_constantcurrent.csv')

avg, std, variance = [],[],[]

for xi in ex_data:
    avg.append(np.mean(ex_data[xi]))
    std_value = np.std(ex_data[xi], ddof=0)
    std.append(std_value)
    variance.append(std_value**2)
ex_data2=[]    
for xi in np.array(ex_data):
    ex_data2.extend(xi)

#print(ex_data2)
#print(std)
cov_matrix_pd = ex_data.cov()
mean_pd = ex_data.mean() 
mean_pd1 = ex_data1.mean()
#print(mean_pd.var())



#print(ex_data1)
#print(list(mean_pd1))

#import plotly.express as px

#fig = px.imshow(cov_matrix_pd)
#fig = px.imshow(cov_matrix_pd,  zmin=-100e-6, zmax=350e-6,)
#fig.show()

corr_matrix_pd = ex_data.corr()

In [3]:
def update_layout_of_graph(fig: go.Figure,title: str = 'Plot')->go.Figure:
    fig.update_layout(
        width=800,
        height=700,
        autosize=False,
        plot_bgcolor='rgba(0,0,0,0)',
        title=title,
        
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                      xaxis_title = 'input values',
                      yaxis_title = 'output values',
                      legend=dict(yanchor="top",
                                  y=0.9,
                                  xanchor="right",
                                  x=0.95),
                      title={
                          'x': 0.5,
                          'xanchor': 'center'
                      })
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
ticks='outside',
)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
ticks='outside', )
    return fig
def line_scatter(
    visible: bool = True,
    x_lines: np.array = np.array([]),
    y_lines: np.array = np.array([]),
    name_line: str = 'Predicted mean',
    showlegend: bool = True,
) -> go.Scatter:
    # Adding the lines
    return go.Scatter(
        visible=visible,
        line=dict(color="green", width=0.5),
        x=x_lines,
        y=y_lines,
        name=name_line,
        showlegend= showlegend
    )

def dot_scatter(
    visible: bool = True,
    x_dots: np.array = np.array([]),
    y_dots: np.array = np.array([]),
    name_dots: str = 'Observed points', s=100, marker='square', zorder=10, alpha = 0.5,
    showlegend: bool = True
) -> go.Scatter:
    # Adding the dots
    return go.Scatter(
        x=x_dots,
        visible=visible,
        y=y_dots,
        mode="markers",
        name=name_dots,
        marker=dict(color='red', size=10, symbol='square'),
        showlegend=showlegend
    )

def uncertainty_area_scatter(
        visible: bool = True,
        x_lines: np.array = np.array([]),
        y_upper: np.array = np.array([]),
        y_lower: np.array = np.array([]),
        name: str = None,
        fillcolor: str = None,
        alpha: float = None,
) -> go.Scatter:

    return go.Scatter(
        visible=visible,
        x=np.concatenate((x_lines, x_lines[::-1])),  # x, then x reversed
        # upper, then lower reversed
        y=np.concatenate((y_upper, y_lower[::-1])),
        fill='toself',
        #fillcolor=f'rgba(0, 0, 255, {alpha})',
        fillcolor=fillcolor,
        line=dict(color='rgba(200,200,200,0)'),
        hoverinfo="skip",
        showlegend=True,
        name= name,
        opacity = 0.3
    )

def add_slider_GPR(figure: go.Figure, parameters):
    figure.data[0].visible = True
    figure.data[1].visible = True

    # Create and add slider
    steps = []
    for i in range(int((len(figure.data) - 1) / 2)):
        step = dict(
            method="update",
            label=f'{parameters[i]: .2f}',
            args=[{
                "visible": [False] * (len(figure.data) - 1) + [True]
            }],
        )
        step["args"][0]["visible"][2 *
                                   i] = True  # Toggle i'th trace to "visible"
        step["args"][0]["visible"][2 * i + 1] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        pad={"t": 50},
        steps=steps,
    )]
    figure.update_layout(sliders=sliders, )
    return figure

In [4]:
class SquaredExponentialKernel:
    def __init__(self, sigma_f: float = 1, length: float = 1):
        self.sigma_f = sigma_f
        self.length = length

    def __call__(self, argument_1: np.array, argument_2: np.array) -> float:
        return float(self.sigma_f**2 *
                     np.exp(-(np.linalg.norm(argument_1 - argument_2)**2) /
                            (2 * self.length**2)))
    
# Helper function to calculate the respective covariance matrices
def cov_matrix(x1, x2, cov_function) -> np.array:
    calculated_cov_matrix = np.array([[cov_function(a, b) for a in x1] for b in x2])
    
    return calculated_cov_matrix

class GPR:
    def __init__(self,
                 data_x: np.array,
                 data_y: np.array,
                 covariance_function=SquaredExponentialKernel(),
                 white_noise_sigma: float = 0):
        
        self.noise = white_noise_sigma
        self.data_x = data_x
        self.data_y = data_y
        self.covariance_function = covariance_function
        

        # Store the inverse of covariance matrix of input (+ machine epsilon on diagonal) since it is needed for every prediction
        self._inverse_of_covariance_matrix_of_input = np.linalg.inv(
            cov_matrix(data_x, data_x, covariance_function) +
            (3e-7 + self.noise) * np.identity(len(self.data_x)))
        
        self._memory = None
        cov_mat_expt = cov_matrix(data_x, data_x, covariance_function)
        

    # function to predict output at new input values. Store the mean and covariance matrix in memory.

    def predict(self, at_values: np.array) -> np.array:
        k_lower_left = cov_matrix(self.data_x, at_values,
                                  self.covariance_function)
        k_lower_right = cov_matrix(at_values, at_values,
                                   self.covariance_function)

        # Mean.
        mean_at_values = np.dot(
            k_lower_left,
            np.dot(self.data_y,
                   self._inverse_of_covariance_matrix_of_input.T).T).flatten()

        # Covariance.
        cov_at_values = k_lower_right - \
            np.dot(k_lower_left, np.dot(
                self._inverse_of_covariance_matrix_of_input, k_lower_left.T))

        # Adding value larger than machine epsilon to ensure positive semi definite
        cov_at_values = cov_at_values + 3e-7 * np.ones(
            np.shape(cov_at_values)[0])

        var_at_values = np.diag(cov_at_values)

        self._memory = {
            'mean': mean_at_values,
            'covariance_matrix': cov_at_values,
            'variance': var_at_values
        }
        return mean_at_values, var_at_values, cov_at_values
    
def plot_GPR(data_x, data_y, model, x, visible=True) -> list:
    mean, variance, dummy2  = model.predict(x)

    std = np.sqrt(model._memory['variance'])
    data = []
    std_dataset = [1,2]
    uncertainty_area=[std*1,variance*20]
    #names = ["std", "variance"]
    names=["With experimental error", "Without experimental error"]
    colors=["blue", "yellow"]
    alpha=[0.1, 0.9]
    for i in range(0, 2):
        y_upper =mean + uncertainty_area[i]
        y_lower = mean - uncertainty_area[i]
        data.append(
            uncertainty_area_scatter(
                x_lines=x,
                y_lower=y_lower,
                y_upper=y_upper,
                #name=f" Experimental error, mean +/- {names[i]}",
                name=names[i],
                visible=visible,
                fillcolor = colors[i],
                alpha=alpha[i]
                
            ))

    data.append(line_scatter(x_lines=x, y_lines=mean, visible=visible))
    #data.append(dot_scatter(x_dots=data_x, y_dots=data_y, visible=visible))
   
    return data

In [5]:
x = np.arange(0.9, 10.1,0.1)

x_values = np.arange(1, 11)   #[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

y=y_values = list(mean_pd)
#print(std)
error = [0]*len(np.array(std))
#print(error)
#y_values = np.array([0.328799153, 0.320079789, 0.326853717, 0.325070659, 0.33154876, 0.320277228, 0.325947031, 0.323554144, 0.325137255, 0.327521048,])
#y= np.array([0.328799153, 0.320079789, 0.326853717, 0.325070659, 0.33154876, 0.320277228, 0.325947031, 0.323554144, 0.325137255, 0.327521048,])
#error=np.array([0.00134765, 0.00319853, 0.002322823, 0.000630938, 0.003041601, 0.00166203, 0.001554742, 0.00177952, 0.003044584, 0.00330258,])


x_ticklabels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
#model = GPR(x_values,
#            y_values,
#            covariance_function=SquaredExponentialKernel(length=0.5),
#            white_noise_sigma=0.1)

#data = plot_GPR(x_ticklabels, y_values, model=model, x=x)

#fig3 = go.FigureWidget(data=data)
    

# we update the y values of our 4 scatter plots 
#@interact(sigma=(0.001, 1, 0.01), length=(0.1, 5, 0.1), noise=(0, 3, 0.1))
#def update(sigma=1.0, length=10, noise=0.0001):
#    with fig3.batch_update():


sigma=0.5
length=1.5
noise=1e-5
model = GPR(x_values,
            y_values,
            covariance_function=SquaredExponentialKernel(
                sigma_f=sigma, length=length),
            white_noise_sigma=noise)

mean, variance, covariance = model.predict(x)

#mean_p, variance_p, covariance_p = model.predict(x_values)
data = plot_GPR(x_ticklabels, y_values, model=model, x=x)
fig3 = go.FigureWidget(data=data)


#x_ticklabels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
fig3.add_scatter(x=x_values, 
                 y=y, mode='markers',  marker=dict(color='red', size=10, symbol='square'), name='Observed value')
                 #error_y = dict(type='data', array=error, visible=True, value=30,), 
                         # name='error',)

fig3 = update_layout_of_graph(fig3)
x_ticklabels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
#fig3.data[1].x = x_ticklabels
fig3.update_layout(yaxis=dict(title='Overpotential / V vs. RHE'), 
                  xaxis=dict(title= 'Catalysts',
                  #gridcolor='grey',
                  #tickmode= 'array',
                  #tickmode= 'linear',
                  tick0= 2050,
                  #dtick=2,
                  tickvals= x_values,
                  ticktext = x_ticklabels)
                 )
fig3.update_layout(legend=dict(yanchor="top", y=1, xanchor="right", x=1), font=dict(size=18), legend_font_family="Arial", legend_font_color="blue", yaxis = dict(range=[0.24, 0.29],  dtick=0.01, showticklabels=True))
fig3.update_xaxes(title_font_family="Arial", title_font_color="black", titlefont=dict(size=26), tickfont=dict(size=20), tickfont_color="black")
fig3.update_yaxes(title_font_family="Arial", title_font_color="black", titlefont=dict(size=26), tickfont=dict(size=20), tickfont_color="black")
#fig3.update_layout(font=dict(size=18), legend_font_family="Arial", legend_font_color="RebeccaPurple")
fig3.update_traces(visible=False, selector=dict(name='Predicted mean'))
fig3

FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': 'blue',
              'hoverinfo': '…

In [6]:
x = np.arange(0.9, 10.1,0.1)

x_values = np.arange(1, 11)   #[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

y=y_values = list(mean_pd1)

error = [0]*len(np.array(std))

x_ticklabels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']


sigma=1.2
length=1.5
noise=1e-5
model = GPR(x_values,
            y_values,
            covariance_function=SquaredExponentialKernel(
                sigma_f=sigma, length=length),
            white_noise_sigma=noise)

mean, variance, covariance = model.predict(x)

#mean_p, variance_p, covariance_p = model.predict(x_values)
data = plot_GPR(x_ticklabels, y_values, model=model, x=x)
fig3 = go.FigureWidget(data=data)


fig3.add_scatter(x=x_values, 
                 y=y, mode='markers',  marker=dict(color='red', size=10, symbol='square'), name='Observed value')
                

fig3 = update_layout_of_graph(fig3)
x_ticklabels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
fig3.update_layout(yaxis=dict(title='Current density / mAcm<sup>-2</sup>'), 
                  xaxis=dict(title= 'Catalysts',
                  tick0= 2050,                  
                  tickvals= x_values,
                  ticktext = x_ticklabels)
                 )
fig3.update_layout(legend=dict(yanchor="top", y=1, xanchor="right", x=1), font=dict(size=18), legend_font_family="Arial", legend_font_color="blue", yaxis = dict(range=[0.07, 0.18],  dtick=0.02, showticklabels=True))
fig3.update_xaxes(title_font_family="Arial", title_font_color="black", titlefont=dict(size=26), tickfont=dict(size=20), tickfont_color="black")
fig3.update_yaxes(title_font_family="Arial", title_font_color="black", titlefont=dict(size=26), tickfont=dict(size=20), tickfont_color="black")
#fig3.update_layout(font=dict(size=18), legend_font_family="Arial", legend_font_color="RebeccaPurple")
fig3.update_traces(visible=False, selector=dict(name='Predicted mean'))
fig3

FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': 'blue',
              'hoverinfo': '…

In [7]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, widgets
from plotly.validators.scatter.marker import SymbolValidator
from sklearn.metrics import r2_score
import math

# Import data
ex_data1 = pd.read_csv('Pt_data_constantpotential.csv')
ex_data = pd.read_csv('Pt_data_constantcurrent.csv')

# Calculate mean, std, and variance
avg, std, variance = [],[],[]
for xi in ex_data:
    avg.append(np.mean(ex_data[xi]))
    std_value = np.std(ex_data[xi], ddof=0)
    std.append(std_value)
    variance.append(std_value**2)

ex_data2=[]    
for xi in np.array(ex_data):
    ex_data2.extend(xi)

cov_matrix_pd = ex_data.cov()
mean_pd = ex_data.mean() 
mean_pd1 = ex_data1.mean()

corr_matrix_pd = ex_data.corr() 

# Define layout function
def update_layout_of_graph(fig: go.Figure,title: str = 'Plot')->go.Figure:
    fig.update_layout(
        width=800,
        height=700,
        autosize=False,
        plot_bgcolor='rgba(0,0,0,0)',
        title=title,
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                      xaxis_title = 'input values',
                      yaxis_title = 'output values',
                      legend=dict(yanchor="top",
                                  y=0.9,
                                  xanchor="right",
                                  x=0.95),
                      title={
                          'x': 0.5,
                          'xanchor': 'center'
                      })
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                     ticks='outside',)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                     ticks='outside', )
    return fig

# Define scatter functions
def line_scatter(
    visible: bool = True,
    x_lines: np.array = np.array([]),
    y_lines: np.array = np.array([]),
    name_line: str = 'Predicted mean',
    showlegend: bool = True,
) -> go.Scatter:
    # Adding the lines
    return go.Scatter(
        visible=visible,
        line=dict(color="green", width=2),
        x=x_lines,
        y=y_lines,
        name=name_line,
        showlegend= showlegend
    )

def dot_scatter(
    visible: bool = True,
    x_dots: np.array = np.array([]),
    y_dots: np.array = np.array([]),
    name_dots: str = 'Observed points',
    s=100,
    marker='square',
    zorder=10,
    alpha = 0.5,
    showlegend: bool = True
) -> go.Scatter:
    # Adding the dots
    return go.Scatter(
        x=x_dots,
        visible=visible,
        y=y_dots,
        mode="markers",
        name=name_dots,
        marker=dict(color='red', size=10, symbol='square'),
        showlegend=showlegend
    )

def uncertainty_area_scatter(
        visible: bool = True,
        x_lines: np.array = np.array([]),
        y_upper: np.array = np.array([]),
        y_lower: np.array = np.array([]),
        name: str = None,
        fillcolor: str = None,
        alpha: float = None,
) -> go.Scatter:

   


SyntaxError: unexpected EOF while parsing (<ipython-input-7-4a3fd5880fee>, line 107)